In [126]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [127]:
def get_cars(num_cars, verbose, path, sleep_time):
    
    '''Gathers cars as a dataframe, scraped from Autotrader'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
   
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    wait = WebDriverWait(driver, 5)
    driver.set_window_size(1120, 1000)

    url = 'https://www.autotrader.ca/cars/bc/vancouver'
    driver.get(url)
    cars = []
    ########Click to enter first page of cars sortlisted for a location#########################
    
    time.sleep(sleep_time)
    try:
        driver.find_element_by_class_name("result-title.click").click()
    except ElementClickInterceptedException:
        pass



#########################################
    while len(cars) < num_cars: #If true, should be still looking for new cars.
        
        #Going through each car details in this page
        
        collected_successfully = False
            
        while not collected_successfully:
            try:
                car_title  = driver.find_element_by_class_name('hero-title').text
                car_location = driver.find_element_by_class_name('hero-location').text  #enters location and distance
                car_price = driver.find_element_by_class_name('hero-price').text
                car_dealer = driver.find_element_by_class_name("di-info-wrapper.text-break").text
                car_contact = driver.find_element_by_xpath('//*[@id="dealerInfo"]/div[2]').text
                collected_successfully = True
            except:
                time.sleep(5)
                
                
                ## attempt to click read more. didn't work. Moving on!
                
#             try:
#                 driver.find_element_by_class_name('ng-star-inserted').click()
#                 print("found")
#             except NoSuchElementException:
#                 pass
            
            try:
                car_description = driver.find_element_by_xpath('//*[@id="descriptionWidget"]/div/div/div[1]/collapsible-container/div/div[1]').text
            except:
                car_description = -1
        
        ##Specifications
        ##click for show more
            try:
#                 driver.find_element_by_xpath('//*[@id="specificationWidget"]/div/div[2]').click()
                wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="specificationWidget"]/div/div[2]'))).click()
                print("found")
            except NoSuchElementException:
                pass
            
            try:
                car_specs = driver.find_element_by_class_name("col-xs-12.card-body.list-container.closed").text
            except:
                car_specs = -1
            
            
                
                
        cars.append({"Car Title" : car_title,
                    "Location" : car_location,
                    "Price" : car_price,
                    "Dealer" : car_dealer,
                     "contact" : car_contact,
                    "Description" : car_description,
                    "Specifications" : car_specs})
            #add car to cars

        #Clicking on the "next page" button
        try:
            driver.find_element_by_id('nextMobileAdLink').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of cars. Needed {}, got {}.".format(num_cars, len(cars)))
            break

    return pd.DataFrame(cars)  #This line converts the dictionary object into a pandas DataFrame.

In [128]:
#This line will open a new chrome window and start the scraping.
path = 'C:\webdrivers\chromedriver'
csv_file_path = r'C:\Users\ajy00\Desktop\Scrap_analyze_proj\Glassdoor_webscrape.csv'
df = get_cars(2, False, path, 2)
# df.to_csv(csv_file_path, index = False)
df

ElementClickInterceptedException: Message: element click intercepted: Element <div _ngcontent-rsn-c96="" class="card-cta ng-star-inserted">...</div> is not clickable at point (387, 730). Other element would receive the click: <p _ngcontent-rsn-c147="" id="navigationTitle" class="ng-star-inserted">...</p>
  (Session info: chrome=90.0.4430.93)


In [108]:
df.to_csv(csv_file_path, index = False)